In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import math
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from filterpy.kalman import KalmanFilter

In [2]:
#importing the clean dataset
df = pd.read_csv('final_data.csv', index_col = 'Date', parse_dates = True)
df

,Ta (°C),Tm (°C),I (W/m2),Month,Day,Time,Season,P (kW)
Date,,,,,,,,
2012-03-06 03:55:00,4.900000,1.300000,0.000000,3,6,3,1,0.000000
2012-03-06 04:55:00,4.705500,1.424667,0.000000,3,6,4,1,0.000000
2012-03-06 05:55:00,4.750000,1.219000,24.314473,3,6,5,1,14.752650
2012-03-06 06:55:00,5.078667,2.003167,174.397848,3,6,6,1,127.163083
2012-03-06 07:55:00,7.105000,5.859667,371.125332,3,6,7,1,302.455700
...,...,...,...,...,...,...,...,...
2013-12-30 16:51:00,13.194333,10.606667,8.329888,12,30,16,0,6.246400
2013-12-30 17:51:00,11.828833,8.566167,0.000000,12,30,17,0,0.000000
2013-12-30 18:51:00,10.951000,7.949333,0.000000,12,30,18,0,0.000000


In [3]:
df.describe()

,Ta (°C),Tm (°C),I (W/m2),Month,Day,Time,Season,P (kW)
count,11488.000000,11488.000000,11488.000000,11488.000000,11488.000000,11488.000000,11488.000000,11488.000000
mean,20.321270,23.790647,284.476799,6.998520,15.456041,12.430014,1.646501,216.872690
std,8.316108,14.337396,315.537087,3.214929,8.514276,5.191415,1.063634,244.484106
min,-1.488000,-7.644500,0.000000,1.000000,1.000000,3.000000,0.000000,0.000000
25%,14.086667,12.979625,0.438084,4.000000,8.000000,8.000000,1.000000,0.016412
50%,19.944667,21.146000,149.092891,7.000000,15.000000,12.000000,2.000000,105.502533
75%,26.297042,34.059125,532.998233,10.000000,23.000000,17.000000,3.000000,414.003819
max,43.532833,70.138833,1071.130972,12.000000,31.000000,21.000000,3.000000,848.660950


In [4]:
print(df.index.dtype)

datetime64[ns]


In [5]:
#splitting the data into training and testing
train=df.iloc[:-24]
test=df.iloc[-24:]

In [6]:
#splitting the data into training and testing
lr_train = train.iloc[:-1000]
lr_test = train.iloc[-1000:]

FEATURES = ['Ta (°C)', 'Tm (°C)', 'I (W/m2)', 'Time', 'Day', 'Month', 'Season']
TARGET = 'P (kW)'

X_train = lr_train[FEATURES]
y_train = lr_train[TARGET]

X_test = lr_test[FEATURES]
y_test = lr_test[TARGET]

In [7]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(10464, 7)
(10464,)
(1000, 7)
(1000,)


In [8]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [9]:
y_pred = lr.predict(X_test)

r2 = r2_score(y_test, y_pred)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
print('R-squared: %.2f' % r2)
print('RMSE: %.2f' % rmse)

R-squared: 0.97
RMSE: 25.44


In [10]:
dfta =pd.DataFrame()
dfta['Ta (°C)'] = train['Ta (°C)']
dftm =pd.DataFrame()
dftm['Tm (°C)'] = train['Tm (°C)']
dfi =pd.DataFrame()
dfi['I (W/m2)'] = train['I (W/m2)']

In [11]:
scalerta = MinMaxScaler()
scalertm = MinMaxScaler()
scaleri = MinMaxScaler()

In [12]:
scalerta.fit(dfta)
scalertm.fit(dftm)
scaleri.fit(dfi)

scaled_dfta = scalerta.transform(dfta)
scaled_dftm = scalertm.transform(dftm)
scaled_dfi = scaleri.transform(dfi)

In [13]:
n_input = 24
n_features = 1
generatorta = TimeseriesGenerator(scaled_dfta, scaled_dfta, length = n_input, batch_size = 1)
generatortm = TimeseriesGenerator(scaled_dftm, scaled_dftm, length = n_input, batch_size = 1)
generatori = TimeseriesGenerator(scaled_dfi, scaled_dfi, length = n_input, batch_size = 1)

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [15]:
modelta = Sequential()
modelta.add(LSTM(100, activation='LeakyReLU', input_shape=(n_input, n_features)))
modelta.add(Dense(1))
modelta.compile(optimizer='adam', loss='mse')

In [16]:
modeltm = Sequential()
modeltm.add(LSTM(100, activation='LeakyReLU', input_shape=(n_input, n_features)))
modeltm.add(Dense(1))
modeltm.compile(optimizer='adam', loss='mse')

In [17]:
modeli = Sequential()
modeli.add(LSTM(100, activation = 'LeakyReLU', return_sequences=True, input_shape=(n_input, n_features)))
modeli.add(Dense(1))
modeli.compile(optimizer='adam', loss='mse')

In [18]:
modelta.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [19]:
modeltm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 100)               40800     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [20]:
modeli.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 24, 100)           40800     
                                                                 
 dense_2 (Dense)             (None, 24, 1)             101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [21]:
modelta.fit(generatorta,epochs=10)

Epoch 1/10
11440/11440 [==============================] - 59s 5ms/step - loss: 0.0023
Epoch 2/10
11440/11440 [==============================] - 59s 5ms/step - loss: 0.0010
Epoch 3/10
11440/11440 [==============================] - 59s 5ms/step - loss: 8.7499e-04
Epoch 4/10
11440/11440 [==============================] - 59s 5ms/step - loss: 8.3682e-04
Epoch 5/10
11440/11440 [==============================] - 59s 5ms/step - loss: 8.0373e-04
Epoch 6/10
11440/11440 [==============================] - 59s 5ms/step - loss: 7.7239e-04
Epoch 7/10
11440/11440 [==============================] - 57s 5ms/step - loss: 7.3144e-04
Epoch 8/10
11440/11440 [==============================] - 57s 5ms/step - loss: 7.2360e-04
Epoch 9/10
11440/11440 [==============================] - 57s 5ms/step - loss: 7.1282e-04
Epoch 10/10
11440/11440 [==============================] - 56s 5ms/step - loss: 6.9257e-04


In [ ]:
modeltm.fit(generatortm,epochs=10)

Epoch 1/10
11440/11440 [==============================] - 57s 5ms/step - loss: 0.0032
Epoch 2/10
11440/11440 [==============================] - 57s 5ms/step - loss: 0.0017
Epoch 3/10
11440/11440 [==============================] - 57s 5ms/step - loss: 0.0016
Epoch 4/10
11440/11440 [==============================] - 57s 5ms/step - loss: 0.0015
Epoch 5/10
11440/11440 [==============================] - 57s 5ms/step - loss: 0.0015
Epoch 6/10
 4447/11440 [==========>...................] - ETA: 34s - loss: 0.0014

In [ ]:
modeli.fit(generatori,epochs=10)

In [ ]:
loss_per_epoch = modelta.history.history['loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch)

In [ ]:
loss_per_epoch = modeltm.history.history['loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch)

In [ ]:
loss_per_epoch = modeli.history.history['loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch)

In [ ]:
test_predictions_ta = []

first_eval_batch = scaled_dfta[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(test)):
    
    # get the prediction value for the first batch
    current_pred = modelta.predict(current_batch)[0]
    
    # append the prediction into the array
    test_predictions_ta.append(current_pred) 
    
    # use the prediction to update the batch and remove the first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
test_predictions_tm = []

first_eval_batch = scaled_dftm[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(test)):
    
    # get the prediction value for the first batch
    current_pred = modeltm.predict(current_batch)[0]
    
    # append the prediction into the array
    test_predictions_tm.append(current_pred) 
    
    # use the prediction to update the batch and remove the first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
test_predictions_i = []

first_eval_batch = scaled_dfi[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(test)):
    
    # get the prediction value for the first batch
    current_pred = modeli.predict(current_batch)[0]
    
    # append the prediction into the array
    test_predictions_i.append(current_pred) 
    
    # use the prediction to update the batch and remove the first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
test_predictions_ta = scalerta.inverse_transform(test_predictions_ta)
test_predictions_tm = scalertm.inverse_transform(test_predictions_tm)
test_predictions_i = scaleri.inverse_transform(test_predictions_i)

In [ ]:
result_ta = pd.DataFrame()
result_ta['Ta True'] = test['Ta (°C)']
result_ta['Ta Predictions'] = test_predictions_ta

result_tm = pd.DataFrame()
result_tm['Tm True'] = test['Tm (°C)']
result_tm['Tm Predictions'] = test_predictions_tm

result_i = pd.DataFrame()
result_i['I True'] = test['I (W/m2)']
result_i['I Predictions'] = test_predictions_i

In [ ]:
result_ta['Ta True'].plot(figsize=(14,5))
result_ta['Ta Predictions'].plot(figsize=(14,5))

In [ ]:
r2_ta = r2_score(result_ta['Ta True'], result_ta['Ta Predictions'])
rmse_ta = math.sqrt(mean_squared_error(result_ta['Ta True'], result_ta['Ta Predictions']))
print('R-squared: %.2f' % r2_ta)
print('RMSE: %.2f' % rmse_ta)

In [ ]:
result_tm['Tm True'].plot(figsize=(14,5))
result_tm['Tm Predictions'].plot(figsize=(14,5))

In [ ]:
r2_tm = r2_score(result_tm['Tm True'], result_tm['Tm Predictions'])
rmse_tm = math.sqrt(mean_squared_error(result_tm['Tm True'], result_tm['Tm Predictions']))
print('R-squared: %.2f' % r2_tm)
print('RMSE: %.2f' % rmse_tm)

In [ ]:
result_i['I True'].plot(figsize=(14,5))
result_i['I Predictions'].plot(figsize=(14,5))

In [ ]:
r2_i = r2_score(result_i['I True'], result_i['I Predictions'])
rmse_i = math.sqrt(mean_squared_error(result_i['I True'], result_i['I Predictions']))
print('R-squared: %.2f' % r2_i)
print('RMSE: %.2f' % rmse_i)

In [ ]:
final_df = pd.DataFrame()
final_df['Ta (°C)'] = result_ta['Ta Predictions'] 
final_df['Tm (°C)'] = result_tm['Tm Predictions']
final_df['I (W/m2)'] = result_i['I Predictions']
final_df['Time'] = test['Time']
final_df['Day'] = test['Day']
final_df['Month'] = test['Month']
final_df['Season'] = test['Season']
final_df.index = pd.to_datetime(test.index)
final_df

In [ ]:
final_power = lr.predict(final_df)

In [ ]:
result_power = pd.DataFrame()
result_power['P True'] = test['P (kW)']
result_power['P Predictions'] = final_power

In [ ]:
result_power['P True'].plot(figsize=(14,5))
result_power['P Predictions'].plot(figsize=(14,5))

In [ ]:
r2_power = r2_score(result_power['P True'], result_power['P Predictions'])
rmse_power = math.sqrt(mean_squared_error(result_power['P True'], result_power['P Predictions']))
print('R-squared: %.2f' % r2_power)
print('RMSE: %.2f' % rmse_power)